# What is PyTorch

### Tensors

In [1]:
from __future__ import print_function
import torch

In [4]:
# construct 5x3 matrix
x = torch.Tensor(5,3)
print(x)


1.00000e-10 *
  0.0000  4.6566  0.0000
  4.6566  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
[torch.FloatTensor of size 5x3]



In [5]:
# construct a randomly initiaized matrix
x = torch.rand(5,3)
print(x)


 0.5530  0.9296  0.7001
 0.9620  0.5339  0.7165
 0.3885  0.5232  0.2129
 0.5998  0.5302  0.8693
 0.2134  0.7459  0.2251
[torch.FloatTensor of size 5x3]



In [9]:
# get its size
print(x.size())

torch.Size([5, 3])


### Operations

In [12]:
# add two tensors
x = torch.rand(5,3)
y = torch.rand(5,3)
print(x+y)
print(x.add(y))


 0.7362  1.1810  0.7894
 0.9359  1.4689  0.6231
 0.9065  0.5030  0.5098
 1.4152  1.4387  1.2503
 0.4314  0.6977  0.7715
[torch.FloatTensor of size 5x3]


 0.7362  1.1810  0.7894
 0.9359  1.4689  0.6231
 0.9065  0.5030  0.5098
 1.4152  1.4387  1.2503
 0.4314  0.6977  0.7715
[torch.FloatTensor of size 5x3]



In [15]:
# adding, giving and output tensor
result = torch.Tensor(5,3)
torch.add(x,y,out=result)
print(result)


 0.7362  1.1810  0.7894
 0.9359  1.4689  0.6231
 0.9065  0.5030  0.5098
 1.4152  1.4387  1.2503
 0.4314  0.6977  0.7715
[torch.FloatTensor of size 5x3]



In [16]:
# add inplace
x.add_(y)
print(x)


 0.7362  1.1810  0.7894
 0.9359  1.4689  0.6231
 0.9065  0.5030  0.5098
 1.4152  1.4387  1.2503
 0.4314  0.6977  0.7715
[torch.FloatTensor of size 5x3]



In [20]:
# numpy notations
print(x[:,0])


 0.7362
 0.9359
 0.9065
 1.4152
 0.4314
[torch.FloatTensor of size 5]



In [25]:
# convert between PyTorch and numpy
x = torch.ones(5,3)
print(x)
print(x.numpy())


 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
[torch.FloatTensor of size 5x3]

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]


In [28]:
x.add_(1)


 2  2  2
 2  2  2
 2  2  2
 2  2  2
 2  2  2
[torch.FloatTensor of size 5x3]

In [33]:
# convert from numpy to pytorch
import numpy as np

x = np.ones((5,3))
torch.from_numpy(x)


 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
[torch.DoubleTensor of size 5x3]

# Autograd: automatic differentiation

In [35]:
import torch
from torch.autograd import Variable

In [40]:
# create a variable
x = Variable(torch.ones(2,2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [43]:
# create a tensor from x, y is created by operation from x, hence it has grad_fn
y = x +2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [51]:
# do more operations on y
z = y*y*3
out = z.mean()
print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



### Gradients

In [52]:
# backpropagate 
out.backward()

In [54]:
# prind gradients d(out)/d(x)
x.grad

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

In [69]:
# do some other crazy stuff
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x*2
while y.data.norm() < 1000:
    y = y*2

print(y)

Variable containing:
 -27.2075
 993.8073
-298.5315
[torch.FloatTensor of size 3]



In [73]:
gradient = Variable(torch.ones(3,1))
y.backward(gradient)

print(x.grad)

Variable containing:
 1024
 1024
 1024
[torch.FloatTensor of size 3x1]



# Neural Networks

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: 
__weight = weight - learning_rate * gradient__

### Define the network

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # If the size is a square, you can specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimensions
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [10]:
# learnable parameters are contained in net.parameters()
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [13]:
# The input to the forward is an autograd.Variable, and so is the output.
input_net = Variable(torch.randn(1,1,32,32))
output_net = net(input_net)
print(output_net)

Variable containing:
 0.1067  0.0000  0.0239  0.0415  0.1108 -0.0298  0.0230  0.0721 -0.0763  0.0296
[torch.FloatTensor of size 1x10]



In [14]:
# Zero the gradient buffers of all parameters and backprops with random gradients:
net.zero_grad()
output_net.backward(torch.randn(1,10))

In [23]:
# Loss function (dummy example)
output = net(input_net)
target = Variable(torch.arange(1,11))
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

Variable containing:
 38.2908
[torch.FloatTensor of size 1]



In [30]:
# update weights
import torch.optim as optim

# create optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in the training loop
optimizer.zero_grad()
output = net(input_net)
loss = criterion(output, target)
loss.backward()
optimizer.step()